In [1]:
import networkx as nx

from OperatorBenchmark import OperatorBenchmark

from typing import List, Dict

from scripts import load_profiling_data,create_profiling_data_ER,separate_keys_and_values,git_branch_commit_push,create_profiling_data_ER_HPC

from scripts_old import load_runMultipleSimpleQWAK, load_runMultipleSimpleQWAK_legacy
from datetime import datetime

import os
import re

from utils.plotTools import plot_qwak

import numpy as np
import cupy as cp
from scipy.linalg import inv, expm
import networkx as nx
import time
import cupyx.scipy.linalg as cpx_scipy
from cupyx.profiler import benchmark
from matplotlib import pyplot as plt
import os
import json
import pickle
from tqdm import tqdm
import subprocess
import random
from qwak_cupy.qwak import QWAK as CQWAK
from qwak.qwak import QWAK as QWAK
from datetime import datetime

Profiling applied to init_operator, config: {'output_path': 'operator_results', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_operator, config: {'output_path': 'operator_results', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_expm_operator, config: {'output_path': 'operator_results_hpc', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to init_operator, config: {'output_path': 'operator_results_hpc', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_operator, config: {'output_path': 'operator_results_hpc', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_expm_operator, config: {'output_path': 'operator_results_hpc', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to init_hiperwal

In [7]:

nMin = 3
nMax = 1000
nList = list(range(nMin, nMax, 1))
pVal = 0.8
samples = 100


SCRIPT_DIR = os.getcwd()
path = os.path.normpath(os.path.join(
    SCRIPT_DIR,
    "Profiling/operator_results"
))
# Verify path exists
if not os.path.exists(path):
    print(f"ERROR: Path not found - {path}")
    print("Current directory contents:", os.listdir(SCRIPT_DIR))
    exit()
# Now load data
result = load_profiling_data(
    path=path,
    method_name="init_operator",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)
result2 = load_profiling_data(
    path=path,
    method_name="build_operator",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)

result3 = load_profiling_data(
    path=path,
    method_name="build_expm_operator",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)

Processing n-values:   0%|          | 0/997 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\jaime\\Documents\\GitHub\\QWAK\\benchmark\\ModuleDev\\Profiling\\operator_results\\n_3\\init_operator-n_3_sample_1_pVal_0_8000_seed_10.prof'

In [8]:
def merge_by_sum(dict_a, dict_b):
    """
    Merge two dictionaries by summing the values of matching keys.
    Assumes both dictionaries have the same keys.
    """
    merged = {}
    for key in dict_a:
        merged[key] = dict_a[key] + dict_b[key]
    return merged


params = {
    'figsize': (12, 8),
    'plot_title' : f'Spectral Decomposition vs Expm',
    'x_label' : 'Graph Size N',
    'y_label' : "Time",
    'legend_labels' : ['spec decomp','expm'],
    'legend_loc': "best",
    'legend_title' : 'Solutions',
    'legend_ncol' : 1,
    # 'color_list' : ['#0000FF', '#008000', '#525252'],
    'color_list' : ['b','g','r'],
    'line_style_list' : ['--', '-','-.' ],
    # 'save_path' : f'Output/CompleteSearch/completePlot_N{N}_NWALKS{numberOfWalks}_S{samples}.png',
    'use_loglog': False,
    'use_cbar' : False,
    'cbar_label' : None, 
    'cbar_ticks' : None,
    'cbar_tick_labels' : None,
    'x_lim' : None,
    # 'x_num_ticks' : 10,
    'y_num_ticks' : 5,
    'x_round_val' : 2,
    'y_round_val' : 3,
    # 'v_line_values' : v_line_values,
    # # 'v_line_style': '--',
    'title_font_size': 20,
    'xlabel_font_size': 22,
    'ylabel_font_size': 22,
    'legend_font_size': 14,
    'legend_title_font_size': 14,
    'tick_font_size': 18,}

merged_result = merge_by_sum(result, result2)
result_keys, result_values = separate_keys_and_values(merged_result)
# result_keys, result_values = separate_keys_and_values(result)
# result2_keys, result2_values = separate_keys_and_values(result2)
result3_keys, result3_values = separate_keys_and_values(result3)



x_value_matrix = [result_keys,result3_keys]
y_value_matrix = [result_values,result3_values]

plot_qwak(x_value_matrix = x_value_matrix, y_value_matrix = y_value_matrix,**params)

NameError: name 'result' is not defined

# QWAK Results - Old Method

In [8]:


# Parameters
nMin = 3
nMax = 1000
nList = list(range(nMin, nMax, 1))
pVal = 0.8
samples = 100
t = 101

# base_dir = 'Datasets/Benchmark-SimpleQWAK-Test_ER2'
# base_dir = 'Datasets/Benchmark-SimpleQWAK-Test_ER2'
# base_dir_cupy = 'Datasets/Benchmark-SimpleQWAK_ER3_CuPy'
base_dir_cupy_970 = 'Datasets/Benchmark-SimpleQWAK_ER-CuPy_970'
base_dir_cupy_3070 = 'Datasets/Benchmark-SimpleQWAK_ER2_CuPy_3070'
base_dir_cupy_3070_2 = 'Datasets/Benchmark-SimpleQWAK_ER2_CuPy_3070_2'
base_dir = 'Datasets/Benchmark-SimpleQWAK_ER-NumPy'


# # Record start datetime
start_datetime = datetime.now()


# runMultipleSimpleQWAK(nList, pVal, t, samples, base_dir, hpc = False)
avg_list = load_runMultipleSimpleQWAK2(nList, pVal, t, base_dir)

avg_list_cupy_970 = load_runMultipleSimpleQWAK2(nList, pVal, t, base_dir_cupy_970)

avg_list_cupy_3070_2 = load_runMultipleSimpleQWAK2(nList, pVal, t, base_dir_cupy_3070_2)
    
# runMultipleSimpleQWAK(nList, pVal, t, samples, base_dir_cupy_3070, hpc = True)
tList_cupy_3070, qwList_cupy_3070,avg_list_cupy_3070 = load_runMultipleSimpleQWAK(nList, pVal, t, base_dir_cupy_3070)

print('CuPy QWAK results computed and saved.')


# print(avg_list_cupy_3070)
print(avg_list)
print(avg_list_cupy_970)
# Print elapsed time
elapsed_time = datetime.now() - start_datetime
print(f"Elapsed time: {elapsed_time}")

plt.plot(nList,avg_list,label='QWAK CPU_NumPy')
plt.plot(nList,avg_list_cupy_3070,label='QWAK GPU_CuPy3070')
plt.plot(nList,avg_list_cupy_3070_2,label='QWAK GPU_CuPy3070_2')
plt.plot(nList,avg_list_cupy_970,label='QWAK GPU_CuPy970')
plt.legend()
plt.show()

def save_time_averages_from_txt(txt_file, nList, pVal, t, base_dir,samples):
    """
    Reads time averages from a text file and saves them in .pkl files, one per N.

    Parameters:
    - txt_file: Path to the text file containing time averages.
    - nList: List of values for N.
    - pVal: Value of p.
    - t: Value of t.
    - base_dir: Base directory where the folders and files will be saved.
    """
    # Read time averages from the text file
    with open(txt_file, 'r') as file:
        time_averages = [float(line.strip()) for line in file]

    # Ensure the number of time averages matches the number of N values
    if len(time_averages) != len(nList):
        raise ValueError("The number of time averages in the text file does not match the number of N values.")

    # Iterate over each value of N
    for idx, n in enumerate(tqdm(nList, desc=f"Processing N values {len(nList)}:{nList[0]}->{nList[-1]}", leave=False)):
        n_dir = os.path.join(base_dir, f"N{n}")
        os.makedirs(n_dir, exist_ok=True)

        # Create the average file path
        avg_file = os.path.join(n_dir, f"AVG-t_P{pVal}_T{t}_sample{samples}.pkl")

        # Save the time average for the current N
        with open(avg_file, 'wb') as f:
            pickle.dump(time_averages[idx], f)

    print("Time averages have been saved successfully.")


# base_dir = 'Datasets/Benchmark-SimpleQWAK_ER-NumPy'

# qwak_times_filename = f'LINUX-simpleQWAKTime_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'
# qwak_times_filename_cupy = f'3070-simpleQWAKTime_CuPy_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'

# qwak_times_file = f'Datasets/Benchmark-SimpleQWAK_ER/' + qwak_times_filename
# qwak_times_file_cupy = f'Datasets/Benchmark-SimpleQWAK_ER/' + qwak_times_filename_cupy

# # Example usage:
# save_time_averages_from_txt(qwak_times_file, nList=nList, pVal=pVal, t=t, samples=samples, base_dir=base_dir)

qwak_times_filename = f'LINUX-simpleQWAKTime_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'
qwak_times_file = f'Datasets/Benchmark-SimpleQWAK_ER/' + qwak_times_filename

qwak_times_filename_cupy_970 = f'LINUX-simpleQWAKTime_CuPy_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'
qwak_times_file_cupy_970 = f'Datasets/Benchmark-SimpleQWAK_ER/' + qwak_times_filename_cupy_970

qwak_times_filename_cupy_3070_2 = f'3070-simpleQWAKTime_CuPy_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'
qwak_times_file_cupy_3070_2 = f'Datasets/Benchmark-SimpleQWAK_ER/' + qwak_times_filename_cupy_3070_2


directory = 'Datasets/Benchmark-SimpleQWAK_ER/'
files = os.listdir(directory)
# print(qwak_times_filename_cupy_3070_2)
# print(files)

# Example usage:
# save_time_averages_from_txt(qwak_times_file_cupy_3070_2, nList=nList, pVal=pVal, t=t, base_dir=base_dir_cupy_3070_2,samples=samples)

NameError: name 'load_runMultipleSimpleQWAK2' is not defined